In [ ]:
#!python yoblo.py --image ..\images\final_to_match\HG_140530_003_FW_N20.jpg -y ..\models\darknet\
#!python yolo.py --image ..\images\NDD20\data\3.jpg -y ..\models\darknet\

In [2]:
import cv2
import json 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import re
import seaborn as sns
import skimage 
from scipy.interpolate import splprep, splev

import time

from skimage.exposure import rescale_intensity
from skimage.io import imread, imsave
from pathlib import Path

# helper methods for feature extraction
from feature_extract import *

In [3]:
IMG_SIZE = 512
IMG_DIR = '../images/'
IMAGES_DIR = IMG_DIR+'final_pigmentation_catalogue_2016'
TARGET_IMAGES_DIR = IMG_DIR+'fin_features'
MAX_FILES = 50

MODEL_DIR="../models/darknet"

In [4]:
def filter(img,thrsh_a,thrsh_b):
    img_in = img.copy()
    xp = [65, 60, 200, 200, 250]
    fp = [0, 45, 200, 200, 0]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    img_tmp = cv2.LUT(img_in, table)
    
    img_tmp = cv2.cvtColor(img_tmp, cv2.COLOR_BGR2GRAY)      


    #blur to reduce noise
#     cv2.blur(img_tmp, (18, 18))    
#     cv2.GaussianBlur(img_tmp, (11, 11), 0)
    
    # apply CLAHE
    img_tmp = claheGray(img_tmp, 1.5,8)
    
    # contrast
    
#     img_tmp = contrast_yt(img_tmp,10,0,255)

    # create binary mask and clean away some spots
    _,img_bin = cv2.threshold(img_tmp, thrsh_a, thrsh_b, cv2.THRESH_BINARY_INV)
#     kernel = np.ones((2, 2), np.uint8)
    img_bin = cv2.erode(img_bin, None, iterations=2)
    img_bin = cv2.dilate(img_bin, None, iterations=2)
#     cv2.imshow("cropped", img_bin)
#     cv2.waitKey(0)
    
    # contour - get the contour from the mask
    img_c, cntr, contours = drawContour(img,img_bin,False,200,6,1,45,250,5,7)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]

    # remove background
    cv2.drawContours(img_in, contours, -1, (0,255,0), -1) 
#     img = cv2.erode(img, None, iterations=1)
#     img = cv2.dilate(img, None, iterations=1)
#     drawContours( src, contours,largest_contour_index, color, CV_FILLED,8,hierarchy);
    return img_in

def filter_mask(img,thrsh_a,thrsh_b,canny_a,canny_b,canny_c):
    img_in = img.copy()
    xp = [65, 60, 200, 200, 250]
    fp = [0, 45, 200, 200, 0]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    img_tmp = cv2.LUT(img_in, table)
    
    img_tmp = cv2.cvtColor(img_tmp, cv2.COLOR_BGR2GRAY)      


    #blur to reduce noise
#     cv2.blur(img_tmp, (18, 18))    
    cv2.GaussianBlur(img_tmp, (13, 13), 0)
    
    # create binary mask and clean away some spots
    _,img_bin = cv2.threshold(img_tmp, thrsh_a, thrsh_b, cv2.THRESH_BINARY_INV)
    kernel = np.ones((3, 3), np.uint8)
    img_bin = cv2.erode(img_bin, kernel, iterations=4)
    img_bin = cv2.dilate(img_bin, kernel, iterations=4)
    
    
       
    # contour - get the contour from the mask
    img_c, cntr, contours,edges = drawContour(img,img_bin,False,200,6,1,canny_a,canny_b,canny_c,7)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]

    # remove background
    cv2.drawContours(img_in, contours, -1, (0,255,0), -1) 
    img_in = cv2.erode(img_in, None, iterations=1)
    img_in = cv2.dilate(img_in, None, iterations=1)
    # create mask
#     img_mask = np.zeros((img_in.shape[0], img_in.shape[1]), np.uint8)
#     cv2.drawContours(img_mask, contours, -1, (255,255,255), -1) 
#     img_mask = cv2.erode(img_mask, None, iterations=1)
#     img_mask = cv2.dilate(img_mask, None, iterations=1)
#     img_mask = (255-img_mask)
#     cv2.imshow("source", img_mask)
    
    return img_in, img_in


In [5]:
# image = cv2.imread("..\\images\\final_to_match\\HG_140530_003_FW_N20.jpg")
image = cv2.imread(IMG_DIR+"NDD20/data/5.jpg")
# cv2.namedWindow("source", cv2.WINDOW_AUTOSIZE)
# imS = cv2.resize(image, (960, 540))  
# cv2.imshow("source", imS)
# cv2.waitKey(0)
# cv2.destroyWindow("source")
# image = resizeAndPad(image, (720,1000), 255)
cv2.namedWindow("source", cv2.WINDOW_AUTOSIZE)

CONFIDENCE = 0.3
THRESHOLD = 0.3

weightsPath = os.path.sep.join([MODEL_DIR, "training/yolov4-dolphin_best.weights"])
configPath = os.path.sep.join([MODEL_DIR, "yolov4-dolphin.cfg"])

img_fins, _ = find_fin(image,weightsPath,configPath,CONFIDENCE,THRESHOLD,IMG_SIZE,0)

print(len(img_fins))

img_original = img_fins[0]
# add border
img = add_border(img_original,5,0)

alpha = 1.0
alpha_max = 500
beta = 0
beta_max = 200
gamma = 1.3
gamma_max = 200


bin_thrsa = 85
bin_thrsb = 240

canny_a = 0.75
canny_b = 0.5
canny_c = 0.05

MASK = False
def basicLinearTransform():
    resa = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
#     print(canny_a)
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    img_correcteda = cv2.LUT(resa, lookUpTable)
    if not MASK:
        img_correctedb, img_mask = filter_mask(img_correcteda,bin_thrsa,bin_thrsb,canny_a,canny_b,3)
    img_corrected = cv2.hconcat([img, img_correctedb])
    cv2.imshow("adjustments", img_corrected)

def on_linear_transform_alpha_trackbar(val):
    global alpha
    alpha = val / 100
    basicLinearTransform()

def on_linear_transform_beta_trackbar(val):
    global beta
    beta = val - 100
    basicLinearTransform()

def on_gamma_correction_trackbar(val):
    global gamma
    gamma = val / 100
    basicLinearTransform()
    
def on_bina_adjust_trackbar(val):
    global bin_thrsa
    bin_thrsa = val
    basicLinearTransform()

def on_binb_adjust_trackbar(val):
    global bin_thrsb
    bin_thrsb = val
    basicLinearTransform()
    
def on_canny_a_adjust_trackbar(val):
    global canny_a
    canny_a = val
    basicLinearTransform()
    
def on_canny_b_adjust_trackbar(val):
    global canny_b
    canny_b = val
    basicLinearTransform()  
    
def on_canny_c_adjust_trackbar(val):
    global canny_c
    canny_c = val
    basicLinearTransform()
    
def setMask(*args):
    pass

img_corrected = np.empty((img.shape[0], img.shape[1]*2, img.shape[2]), img.dtype)
img_gamma_corrected = np.empty((img.shape[0], img.shape[1]*2, img.shape[2]), img.dtype)

cv2.namedWindow("adjustments", cv2.WINDOW_AUTOSIZE)
# cv2.namedWindow('Gamma correction')

alpha_init = int(alpha *100)
cv2.createTrackbar('Alpha gain (contrast)', 'adjustments', alpha_init, alpha_max, on_linear_transform_alpha_trackbar)
beta_init = beta + 100
cv2.createTrackbar('Beta bias (brightness)', 'adjustments', beta_init, beta_max, on_linear_transform_beta_trackbar)
gamma_init = int(gamma * 100)
cv2.createTrackbar('Gamma correction', 'adjustments', gamma_init, gamma_max, on_gamma_correction_trackbar)

bina_init = int(bin_thrsa)
cv2.createTrackbar('Binary Threshold a', 'adjustments', bina_init, 255, on_bina_adjust_trackbar)
binb_init = int(bin_thrsb)
cv2.createTrackbar('Binary Threshold b', 'adjustments', binb_init, 255, on_binb_adjust_trackbar)

# canny_a_init = int(canny_a*100)
# cv2.createTrackbar('Canny Threshold a', 'adjustments', canny_a_init, 255, on_canny_a_adjust_trackbar)
# canny_b_init = int(canny_b*100)
# cv2.createTrackbar('Canny Threshold b', 'adjustments', canny_b_init, 255, on_canny_b_adjust_trackbar)
# canny_c_init = int(canny_c*100)
# cv2.createTrackbar('Canny Threshold c', 'adjustments', canny_c_init, 7, on_canny_c_adjust_trackbar)

#cv2.createButton("Set Maks",setMask,NULL,cv2.QT_PUSH_BUTTON,1)


on_linear_transform_alpha_trackbar(alpha_init)
on_gamma_correction_trackbar(gamma_init)
on_bina_adjust_trackbar(bina_init)
on_binb_adjust_trackbar(binb_init)
# on_canny_a_adjust_trackbar(canny_a_init)
# on_canny_b_adjust_trackbar(canny_b_init)
# on_canny_c_adjust_trackbar(canny_c_init)

cv2.waitKey()
cv2.destroyAllWindows() 


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 2097152 bytes in function 'cv::OutOfMemoryError'


In [ ]:
exit(0)
# image = cv2.imread("..\\images\\final_to_match\\HG_140530_003_FW_N20.jpg")
image = cv2.imread("../images/NDD20/data/5.jpg")
# image = resizeAndPad(image, (720,1000), 255)

CONFIDENCE = 0.3
THRESHOLD = 0.3

weightsPath = os.path.sep.join([MODEL_DIR, "training\\yolov4-dolphin_best.weights"])
configPath = os.path.sep.join([MODEL_DIR, "yolov4-dolphin.cfg"])

img_fins = find_fin(image,weightsPath,configPath,CONFIDENCE,THRESHOLD,IMG_SIZE,0)

print(len(img_fins))
for img in img_fins:
    # make gray
    cv2.imshow("cropped", img)
    cv2.waitKey(0)
    
    # add border
    img = add_border(img,5,0)
    
#     img_cntr = contrast_yt(img,100,0,255,1)
    yt = threshold_yen(img)
#     img_cntr = rescale_intensity(img, (0, yt), (0, 255))
    
    
    alpha = 1.0 # Simple contrast control
    beta = 0    # Simple brightness control
    gamma = 0.5    
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    img_gamma = cv2.LUT(img, lookUpTable)
    #img_gamma = cv2.hconcat([img, res])
    
    cv2.imshow("cropped", img_gamma)
    cv2.waitKey(0)   
    
#     image = image.astype('float32')
    img_tmp = img_gamma
    #contrast stretching
    xp = [65, 80, 200, 200, 250]
    fp = [0, 45, 200, 200, 0]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    img_tmp = cv2.LUT(img_tmp, table)
    
    img_tmp = cv2.cvtColor(img_tmp, cv2.COLOR_BGR2GRAY)      
#     cv2.imshow("cropped", img_tmp)
#     cv2.waitKey(0)
#     cv2.bilateralFilter(img, 3, 50, 50)

    #blur to reduce noise
#     cv2.blur(img_tmp, (18, 18))    
#     cv2.GaussianBlur(img_tmp, (11, 11), 0)
    
    # apply CLAHE
    img_tmp = claheGray(img_tmp, 1.5,8)
    
    # contrast
    
#     img_tmp = contrast_yt(img_tmp,10,0,255)

    # create binary mask and clean away some spots
    _,img_bin = cv2.threshold(img_tmp, 100, 240, cv2.THRESH_BINARY_INV)
#     kernel = np.ones((2, 2), np.uint8)
    img_bin = cv2.erode(img_bin, None, iterations=2)
    img_bin = cv2.dilate(img_bin, None, iterations=2)
#     cv2.imshow("cropped", img_bin)
#     cv2.waitKey(0)
    
    # contour - get the contour from the mask
    img_c, cntr, contours = drawContour(img_gamma,img_bin,False,200,6,1,45,250,5,7)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]

    # remove background
    cv2.drawContours(img_gamma, contours, -1, (0,0,0), -1) 
#     img = cv2.erode(img, None, iterations=1)
#     img = cv2.dilate(img, None, iterations=1)
#     drawContours( src, contours,largest_contour_index, color, CV_FILLED,8,hierarchy);

    cv2.imshow("cropped", img_gamma)
#     cv2.imshow("cntr", cntr)
    cv2.waitKey(0)
    exit
        
#             crop_img = detectFin(crop_img)
# show the output image
# cv2.imshow("Image", image)
# crop_img = resizeAndPad(crop_img, (512,512), 255)
# cv2.imshow("cropped", crop_img)
# cv2.waitKey(0)